# Scraping data from Xendan website

In [9]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
import pandas as pd

In [15]:
data_list=[]
pages=set()
def getLinks(pageUrl):
    html = urlopen(f'https://beenproxies.netlify.app/?destination={pageUrl}')
    bs = BeautifulSoup(html, 'html.parser')

    header=bs.find('div',{'class':"detail-top"}).find('h1').text.strip()
    article=bs.find('div',{'class':"detail-big-text-p"}).text.strip()
    data_list.append({'Header':header,'Article':article,'Link':pageUrl})
    for link in bs.find_all('a', href=re.compile(r'https://www\.(xendan\.org/detailnews/\d+|nrttv\.com/.*)')):
     if 'href' in link.attrs:
        if link.attrs['href'] not in pages:
            #We have encountered a new page
             newPage = link.attrs['href']
             print(newPage)
             pages.add(newPage)
             getLinks(newPage)
getLinks('https://www.xendan.org/detailnews/178566')

https://www.xendan.org/detailnews/178611
https://www.xendan.org/detailnews/178610
https://www.xendan.org/detailnews/178609
https://www.xendan.org/detailnews/178608
https://www.xendan.org/detailnews/178606
https://www.xendan.org/detailnews/178604
https://www.xendan.org/detailnews/178603
https://www.xendan.org/detailnews/178602
https://www.xendan.org/detailnews/178601
https://www.xendan.org/detailnews/178600
https://www.xendan.org/detailnews/178013
https://www.xendan.org/detailnews/178378
https://www.xendan.org/detailnews/178581
https://www.xendan.org/detailnews/178573
https://www.xendan.org/detailnews/178555
https://www.xendan.org/detailnews/178546
https://www.xendan.org/detailnews/178071
https://www.xendan.org/detailnews/178258
https://www.xendan.org/detailnews/178596
https://www.xendan.org/detailnews/178595
https://www.xendan.org/detailnews/178577
https://www.xendan.org/detailnews/178570
https://www.xendan.org/detailnews/178567
https://www.xendan.org/detailnews/177037
https://www.xend

In [13]:

data_list=[]
pages=set()
def getLinks(pageUrl):
    html = urlopen(pageUrl)
    bs = BeautifulSoup(html, 'html.parser')

    header=bs.find('div',{'class':"detail"}).find('a').text.strip()
    article=bs.find('section',{'class':"container"}).text.strip()
    data_list.append({'Header':header,'Article':article,'Link':pageUrl})
    for link in bs.find_all('a', href=re.compile(r'https://www\.nrttv\.com/detail/\d+'or r'https://www\.nrttv\.com/.*')):
     if 'href' in link.attrs:
        if link.attrs['href'] not in pages:
            #We have encountered a new page
             newPage = link.attrs['href']
             print(newPage)
             pages.add(newPage)
             getLinks(newPage)
getLinks('https://nrttv.com/detail/35548')

In [7]:
data_list

[{'Header': 'رێباز حەملان: هەفتەی داهاتوو تەواوکاری لیستی مووچەی مانگی دوو بڵاو دەکرێتەوە',
  'Article': 'سەرەکی\nدەربارە\nپەیوەندی\nئەرشیف\n\n\n\n\n\n\n\nV:11636  - 27500 \xa0\n\nپەخشی ڕاستەوخۆ\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nگەڕان لە ماڵپەڕی NRT\n\n×\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nMENU\n\n\nکوردی\n\n\nBack\n\nسۆرانی\nبادینی\nلاتینی\n\n\nEnglish\nعربي\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\xa0\xa0سەرەکی\nهەواڵ \xa0\n\n\xa0\xa0سیاسیی  \n\n\xa0کوردستان\n\xa0عێراق\n\xa0خۆرھەڵاتی ناوەڕاست\n\xa0جیھان\n\n\n\xa0\xa0ئابوریی\n\xa0\xa0کۆمەڵایەتیی\n\xa0\xa0کولتوریی\n\xa0\xa0زانستیی\n\xa0\xa0تەکنەلۆژیی\n\xa0\xa0جۆراوجۆر\n\n\nڕاپۆرت\xa0\n\n\xa0\xa0کوردستانیی  \n\xa0\xa0عێراقیی\n\xa0\xa0هەرێمیی\n\xa0\xa0نێودەوڵەتیی\n\n\nشیکردنەوە\xa0\n\n\xa0\xa0کوردستانیی  \n\xa0\xa0عێراقیی\n\xa0\xa0هەرێمیی\n\xa0\xa0نێودەوڵەتیی\n\n\nبیروڕا\nبەرنامەکان\nگەلەری \xa0\n\n\xa0\xa0وێنە  \n\xa0\xa0ڤیدیۆ\n\n\nوەرزش\nئێن ئاڕ تی دوو\nوێب تی ڤی\xa0\n\n\xa0\xa0XYZ  \n\xa0\xa0XYZ+\n\xa0\xa0فیچەر\n\n\nسەنتەری پ

In [12]:
scraped_data=pd.DataFrame(data_list)

In [80]:
scraped_data.shape

(42, 3)

In [81]:
scraped_data.to_csv('../scrapedData/xendan.csv')